In [1]:
from sklearn.preprocessing import RobustScaler, StandardScaler
from statsmodels.robust.scale import huber
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import plotly.express as px
import re
import locale
import geopandas
plt.style.use('seaborn')
%matplotlib inline

pd.set_option('display.max_rows', 0)
pd.set_option('display.max_columns', 0)

In [2]:
df_final_pob=pd.read_parquet('../data/main_processed/df_final_pob.parquet')

df_indicadores_pob=pd.read_parquet('../data/main_processed/df_indicadores_pob.parquet')

df_indicadores_pob_pivot=pd.read_parquet('../data/main_processed/df_indicadores_pob_pivot.parquet')

df_final_pob_melt=pd.read_parquet('../data/main_processed/df_final_pob_melt.parquet')

df_final_pob_melt_PC=pd.read_parquet('../data/main_processed/df_final_pob_melt_PC.parquet')

CCAA_CO = pd.read_parquet('../data/main_processed/CCAA_CO.parquet')
MUNI_CO = pd.read_parquet('../data/main_processed/MUNI_CO.parquet')
PROV_CO = pd.read_parquet('../data/main_processed/PROV_CO.parquet')

df_final_pob_dropdown=pd.read_parquet('../data/main_processed/df_final_pob_dropdown.parquet')

df_final_pob_dropdown_c=pd.read_parquet('../data/main_processed/df_final_pob_dropdown_c.parquet')

df_final_pob_poblaciontext=pd.read_parquet('../data/main_processed/df_final_pob_poblaciontext.parquet')


# %%timeit

In [6]:
df_final_pob.columns

Index(['CCAA', 'Provincia', 'Tipo Ente Principal', 'Nombre Ente Principal',
       'Código Ente Principal', 'Abastecimiento de agua potable',
       'Abastos, mercados, lonjas ', 'Acceso a los núcleos de población',
       'Actividades funerarias', 'Alcantarillado', 'Alumbrado público',
       'Biblioteca pública', 'Cementerio', 'Comercio ambulante',
       'Conservación y rehabilitación de la edificación',
       'Conservación, mantenimiento y vigilancia de los edificios destinados a educación',
       'Cooperar con las Administraciones educativas en la obtención de solares',
       'Equipamientos culturales',
       'Evacuación y tratamiento de aguas residuales', 'Ferias ',
       'Información y promoción de la actividad turística de interés y ámbito local',
       'Infraestructura viaria y otros equipamientos de titularidad de la entidad local',
       'Instalaciones de ocupación del tiempo libre',
       'Instalaciones deportivas', 'Instalaciones deportivas de uso público',
       